# library

In [26]:
import pickle


import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel

# load data

In [3]:
with open('/content/drive/MyDrive/train_data.pickle', 'rb') as f:

    data = pickle.load(f)

In [4]:
print(len(data))

288230


In [13]:
data[4]

{'knowledge': ['처음 이유식을 시작하면 한 달간은 하루에 한 번 먹이는 것으로 충분하다.',
  '이유식을 처음 먹는 날은 모유나 분유를 조금 먹인 후 1작은술에 쌀죽을 반 정도 담아 먹인 다음 다시 모유나 분유를 먹인다.'],
 'query': '그럼 이제 슬슬 시작해볼까봐요. 다들 먹이는 양이 다르던데 어느정도 먹이면 좋을까요?',
 'answer': '처음 이유식을 시작하면 한 달간은 하루에 한 번 먹이는 거로 충분해요. 모유나 분유 외의 것을 처음 접하는 것이기 때문에 처음 먹는 날은 모유나 분유를 조금 먹인 다음 쌀미음 반작은술 정도 먹이고 다시 모유나 분유를 먹여보세요.'}

# preprocess

In [14]:
d = data[4]

In [15]:
knowledge = ' '.join(d['knowledge'])

'처음 이유식을 시작하면 한 달간은 하루에 한 번 먹이는 것으로 충분하다. 이유식을 처음 먹는 날은 모유나 분유를 조금 먹인 후 1작은술에 쌀죽을 반 정도 담아 먹인 다음 다시 모유나 분유를 먹인다.'

In [18]:
s = f"질문: {d['query']}\n지식: {knowledge}\n대답: "

print(s)

질문: 그럼 이제 슬슬 시작해볼까봐요. 다들 먹이는 양이 다르던데 어느정도 먹이면 좋을까요?
지식: 처음 이유식을 시작하면 한 달간은 하루에 한 번 먹이는 것으로 충분하다. 이유식을 처음 먹는 날은 모유나 분유를 조금 먹인 후 1작은술에 쌀죽을 반 정도 담아 먹인 다음 다시 모유나 분유를 먹인다.
대답: 


In [19]:
o = d['answer']

print(o)

처음 이유식을 시작하면 한 달간은 하루에 한 번 먹이는 거로 충분해요. 모유나 분유 외의 것을 처음 접하는 것이기 때문에 처음 먹는 날은 모유나 분유를 조금 먹인 다음 쌀미음 반작은술 정도 먹이고 다시 모유나 분유를 먹여보세요.


In [21]:
f"지식: {' '.join(data[0]['knowledge'])}"

'지식: '

In [22]:
def preprocess(data):

    preprocessed_data = []

    for i in range(len(data)):

        d = data[i]

        knowledge = ' '.join(d['knowledge'])

        s = f"질문: {d['query']}\n지식: {knowledge}\n대답: "
        o = d['answer']

        preprocessed_data.append((i,s,o))

    return preprocessed_data


In [23]:
preprocessed_train_data = preprocess(data)

print(len(preprocessed_train_data))

288230


In [24]:
preprocessed_train_data[0]

(0,
 '질문: 저희 애가 슬슬 이유식을 시작해야 할 것 같은데 언제 시작하면 좋을 지 모르겠어요.\n지식: \n대답: ',
 '아기가 지금 몇 개월이나 됐죠?')

# dataset

In [27]:
tokenizer = PreTrainedTokenizerFast.from_pretrained('byeongal/Ko-DialoGPT')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.52M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/109 [00:00<?, ?B/s]

In [29]:
tokenizer(preprocessed_train_data[0][1])

{'input_ids': [24454, 401, 9265, 8806, 9831, 6824, 11340, 7880, 10175, 9562, 9347, 9685, 9337, 9031, 9239, 7220, 12668, 9347, 9511, 34019, 9027, 42138, 14782, 18910, 401, 9051, 7198, 7192, 401, 739], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [31]:
class DialoDataset(Dataset):
    def __init__(self, dataset, tokenizer):

        tokenizer = tokenizer
        self.query = [tokenizer(d[1]) for d in dataset]
        self.answer = [tokenizer(d[2]) for d in dataset]

    def __getitem__(self, i):
        return (self.query[i],self.answer[i])

    def __len__(self):
        return len(self.answer)

In [32]:
train_dataset = DialoDataset(preprocessed_train_data,tokenizer)

# inference test

In [33]:
model = GPT2LMHeadModel.from_pretrained('byeongal/Ko-DialoGPT')

config.json:   0%|          | 0.00/908 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/513M [00:00<?, ?B/s]

In [34]:
train_dataset[0]

({'input_ids': [24454, 401, 9265, 8806, 9831, 6824, 11340, 7880, 10175, 9562, 9347, 9685, 9337, 9031, 9239, 7220, 12668, 9347, 9511, 34019, 9027, 42138, 14782, 18910, 401, 9051, 7198, 7192, 401, 739], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]},
 {'input_ids': [25349, 9782, 9774, 37871, 9185, 39417, 8234, 406], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]})

In [43]:
t = torch.tensor(train_dataset[0][0]['input_ids']).unsqueeze(0)

t.shape

torch.Size([1, 30])

In [47]:
o = model.generate(t,max_length = 100)

In [ ]:
o.tolist()[0]

In [52]:
tokenizer.decode(o.tolist()[0])

'질문: 저희 애가 슬슬 이유식을 시작해야 할 것 같은데 언제 시작하면 좋을 지 모르겠어요.\n지식: \n대답: <unk>가 먼저 말 시작하면 난 <unk>가 먼저 할 거예요.</s>'